In [9]:
%matplotlib inline

In [10]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import math
import re #using regex

In [11]:
#df = pd.read_csv("../../X0_raw_data/log2_tab.csv")
#df.columns
#renamed_col = [x.split("-")[0]+"_"+x.split(" ")[1].split("_")[0]+
# "_"+ x.split("-")[1].split(" ")[0] for x in df.columns if "B6" in x or "TC" in x]
#df.columns = ["MID"] + renamed_col + ['pool 1', 'pool 2'] 

#df.to_csv("../../X0_raw_data/log2_tab_renamed.csv", index = False)

In [12]:
df = pd.read_csv("../../X2_log2transform/Jeff-featab_rmTfh5_log2.csv")

In [13]:
df.columns = ["Metabolite"] + df.columns[1:].tolist()

In [14]:
df.columns

Index(['Metabolite', 'II_SC_1_Tcells', 'II_SC_2_Tcells', 'II_SC_3_Tcells',
       'II_SC_4_Tcells', 'II_SC_6_Tfhcells', 'II_SC_7_Tfhcells',
       'II_SC_8_Tfhcells', 'II_SC_9_Th1cells', 'II_SC_10_Th1cells',
       'II_SC_11_Th1cells', 'II_SC_12_Th1cells', 'II_SC_13_Th17cells',
       'II_SC_14_Th17cells', 'II_SC_15_Th17cells', 'II_SC_16_Th17cells'],
      dtype='object')

In [15]:
#Set your Exp_id & condition
Exp_id = ""
cond = ["Tcells","Tfhcells","Th1cells","Th17cells"]

In [18]:
[x for x in df.columns if Exp_id in x and re.search(cond[1], x)] #using regex

['II_SC_6_Tfhcells', 'II_SC_7_Tfhcells', 'II_SC_8_Tfhcells']

In [19]:
res_df = pd.DataFrame(df.iloc[:,0])

for i in range(df.shape[0]):
    try:
        f,p = stats.f_oneway( [float(x) for x in (df.loc[i,[x for x in df.columns if re.search(cond[0], x)]])], 
                              [float(x) for x in (df.loc[i,[x for x in df.columns if  re.search(cond[1], x)]])],
                              [float(x) for x in (df.loc[i,[x for x in df.columns if  re.search(cond[2], x)]])],
                              [float(x) for x in (df.loc[i,[x for x in df.columns if  re.search(cond[3], x)]])])
        if math.isnan(p) is True:
            res_df.loc[i,"ftest_pval" ] = 1
            res_df.loc[i,"ftest_fscore"] = 0
        else:
            res_df.loc[i,"ftest_pval" ] = p
            res_df.loc[i,"ftest_fscore"] = f
    except Exception as e:
        print(e)
        res_df.loc[i,"ftest_pval"] = 1
        res_df.loc[i,"ftest_fscore"] = 0

In [20]:
res_df.to_csv("ftest4" + Exp_id + "_" + "vs".join([x.replace(".*","_") for x in cond]) + ".txt", sep='\t', index = None)

-----------

KW-test

In [42]:
#Set your Exp_id & condition
Exp_id = ""
cond = ["PUF1","cobA"]

res_df = df.iloc[:,0:3]
try:
    for i in range(df.shape[0]):
        t,p = stats.kruskal( [float(x) for x in (df.loc[i,[x for x in df.columns if Exp_id in x and cond[0] in x]])], 
                             [float(x) for x in (df.loc[i,[x for x in df.columns if Exp_id in x and cond[1] in x]])],
                             nan_policy = 'omit')
        res_df.loc[i,"htest_pval_" + Exp_id + "_" + "vs".join(cond) ] = p
        res_df.loc[i,"htest_tscore_" + Exp_id  + "_" + "vs".join(cond)] = t
        if isinstance(t,str):
            print("yes")
except Exception as e:
    print(e)        
    
res_df["htest_pval_" + Exp_id + "_" + "vs".join(cond)].replace(np.nan, 1.0 , inplace=True)
res_df["htest_tscore_" + Exp_id + "_" + "vs".join(cond)].replace(np.nan, 1.0 , inplace=True)

    
cols = res_df.columns.tolist()

new_cols = cols[1:] + [cols[0]]
new_cols

res_df_new = res_df[new_cols]

res_df_new.to_csv("htest4mcg_" + Exp_id + "_" + "vs".join(cond) + ".txt", sep='\t', index = None)

U-Whiney test

In [47]:
#Set your Exp_id & condition
Exp_id = "_6_"
cond = ["PUF1","cobA"]

res_df = df.iloc[:,0:3]
try:
    for i in range(df.shape[0]):
        t,p = stats.mannwhitneyu( [float(x) for x in (df.loc[i,[x for x in df.columns if Exp_id in x and cond[0] in x]])], 
                                  [float(x) for x in (df.loc[i,[x for x in df.columns if Exp_id in x and cond[1] in x]])],
                                  use_continuity=True, alternative='two-sided')
        res_df.loc[i,"utest_pval_" + Exp_id + "_" + "vs".join(cond) ] = p
        res_df.loc[i,"utest_tscore_" + Exp_id  + "_" + "vs".join(cond)] = t
        if isinstance(t,str):
            print("yes")
except Exception as e:
    print(e)        
    
res_df["utest_pval_" + Exp_id + "_" + "vs".join(cond)].replace(np.nan, 1.0 , inplace=True)
res_df["utest_tscore_" + Exp_id + "_" + "vs".join(cond)].replace(np.nan, 1.0 , inplace=True)

    
cols = res_df.columns.tolist()

new_cols = cols[1:] + [cols[0]]
new_cols

res_df_new = res_df[new_cols]

res_df_new.to_csv("utest4mcg_" + Exp_id + "_" + "vs".join(cond) + ".txt", sep='\t', index = None)

All numbers are identical in mannwhitneyu
